In [ ]:
# from preprocessing.data_collection.multipleye_data_collection import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path

## Pre-processing MultiplEYE Data

In [2]:
data_collection_name = "MultiplEYE_SQ_CH_Zurich_1_2025"
# data_collection_name = 'MultiplEYE_SL_SI_Ljubljana_1_2025'

If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [3]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

# MultipleyeDataCollection.create_from_data_folder(data_folder_path)


In [4]:
multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)

preprocessed_data_folder = this_repo / "preprocessed_data" / data_collection_name
preprocessed_data_folder.mkdir(parents=True, exist_ok=True)

Folder test_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder data_piloting_stimuli_MultiplEYE_SQ_CH_Zurich_1_2025participant_id_1_to_5 does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder pilot_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.


Converting EDF to ASC: 100%|██████████| 9/9 [00:00<00:00, 2873.25it/s]


Preparing session 011_SQ_CH_1_ET1: 100%|██████████| 9/9 [00:12<00:00,  1.40s/it]                    


In [5]:
multipleye_sq

Title	MultiplEYE_SQ_CH_Zurich_1_2025
Dataset_type	MultiplEYE
Number_of_sessions	9
Number_of_pilots	0
Tested_language	SQ
Country	CH
Year	2025
Number of eye-tracking (ET) sessions per participant	1

In [6]:
sessions = [s for s in multipleye_sq]
sess = sessions[0]
idf = sess.session_identifier

## Creating Gaze Frame from ASCII File

In [7]:
asc = sess.asc_path
output_folder = preprocessed_data_folder / idf
output_folder.mkdir(parents=True, exist_ok=True)

In [8]:
gaze = peyepeline.load_gaze_data(
    asc_file=asc,
    lab_config=sess.lab_config,
    session_idf=idf,
    trial_cols=constants.TRIAL_COLS,
)

In [ ]:
preprocessing.save_raw_data(output_folder / "raw_data", sess.session_identifier, gaze)

## Coordinate and Velocity Preprocessing

Eye movements are recorded in screen pixel coordinates, which depend on stimulus size and monitor setup. To compare gaze behavior across participants, screens, or datasets, it is standard to convert pixel positions 
into **degrees of visual angle (dva)**. Next, we compute **gaze velocity**, which allows us to detect saccades and distinguish them from fixations.

In [10]:
preprocessing.preprocess_gaze(gaze)

## Detect Events and Compute Their Properties

Eye-tracking data are typically segmented into events, i.e. `fixations` and `saccades`. Fixations represent moments when the eyes remain relatively still, allowing visual information to be processed, while saccades are the rapid movements between fixations that reposition the gaze. Detecting these events and computing their properties, such as `dispersion`, fixation `duration`, saccade `amplitude`, and `peak velocity`, provides the foundation for analyzing visual behavior and understanding how participants explore a stimulus.

### Fixations

We can detect fixations by applying the `I-VT` or the `I-DT` method.

The **I-VT (Velocity-Threshold Identification)** method distinguishes fixation and saccade points based on their point-to-point velocities. Each point is classified as a fixation if its velocity is below the specified threshold. Consecutive fixation points are then merged into a single fixation. A threshold of 20 degrees/second is commonly used as a default maximum value. Read more about [the IVT algorithm in the documentation](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.ivt.html) 

The **I-DT (Dispersion-Threshold Identification)** method finds fixations by grouping consecutive points within a maximum separation (dispersion) threshold and a minimum duration threshold. The algorithm slides a moving window across the data: if the dispersion within the window is below the threshold, the window represents a fixation and is gradually expanded until the dispersion exceeds the threshold.
Read more about [our implementation of the IDT method](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.idt.html).

We use the `I-VT` algorithm with the following key deafault parameters:
- `minimum duration`: 100 ms 
- `velocity threshold`: 20.0

Such properties as `location`, containing the centroid coordinates of each fixation, and `dispersion` will also be calculated.

In [11]:
preprocessing.detect_fixations(
    gaze,
)

### Saccades

Saccades are rapid eye movements that shift the point of fixation from one location to another. We detect saccades (or micro-saccades) from the velocity sequence of gaze data using the [microsaccades algorithm](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.microsaccades.html#pymovements.events.detection.microsaccades). This algorithm implements a noise-adaptive velocity threshold, meaning that the detection threshold automatically scales with the noise level of the velocity signal. Such properties as `amplitude` and `peak velocity` of the detected saccades will also be calcuated.

The key default parameters are:
- `threshold_factor`: Multiplier used to determine the velocity threshold relative to the noise level of the signal. The default value is 6. A higher factor makes the algorithm more conservative (detects fewer saccades), while a lower factor makes it more sensitive.
- `minimum_duration`: Defines how long a velocity peak must persist to be classified as a saccade. The duration is expressed in the same units as timesteps. If no timesteps are provided, the value refers to the number of samples (default = 6), which corresponds to about 12 ms at a 500 Hz sampling rate. Shorter events are ignored as noise. 

In [12]:
preprocessing.detect_saccades(
    gaze,
)

In [13]:
preprocessing.map_fixations_to_aois(
    gaze,
    sess.stimuli,
)

In [ ]:
gaze.save(output_folder / 'preprocessed_gaze', save_events=True, save_samples=True, verbose=2)

## Calculate Reading Measures

In [15]:
from preprocessing.metrics.words import all_tokens_from_aois, mark_skipped_tokens, repair_word_labels
from preprocessing.metrics.fixations import annotate_fixations
from preprocessing.metrics.reading_measures import build_word_level_table

import polars as pl

trial = 4
trial_label = "trial_" + str(trial)
aois = sess.stimuli[trial].text_stimulus.aois

In [16]:
# add word label to blank spaces between words in AOIs
aois_clean = repair_word_labels(aois)

# collect all words from AOIs for the given trial
all_tokens = all_tokens_from_aois(aois_clean, trial=trial_label)

### Fixation-based Metrics

In [17]:
# create a fixation table
fixation_table = annotate_fixations(gaze.events.frame)
fixation_table

trial,page,fixation_id,onset,word_idx,char_idx,char,run_id,is_first_pass,duration,word,prev_word_idx,next_word_idx,is_reg_in,is_reg_out,is_first_fix
str,str,u32,i64,i64,i64,str,i64,bool,i64,str,i64,i64,bool,bool,bool
"""PRACTICE_trial_1""","""page_1""",0,2630429,1,5,"""t""",1,true,126,"""https://sq.wikipedia.org/wiki/…",null,1,null,false,true
"""PRACTICE_trial_1""","""page_1""",1,2630834,1,6,"""t""",1,true,540,"""https://sq.wikipedia.org/wiki/…",1,3,false,false,false
"""PRACTICE_trial_1""","""page_1""",2,2634207,3,45,"""i""",2,true,108,"""Wikipedia,""",1,4,false,false,true
"""PRACTICE_trial_1""","""page_1""",3,2635216,4,57,"""k""",3,true,200,"""enciklopedia""",3,7,false,false,true
"""PRACTICE_trial_1""","""page_1""",4,2636548,7,74,"""n""",4,true,306,"""Hëna""",4,8,false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""trial_9""","""page_9""",4518,9507281,101,639,"""a""",33,true,146,"""përmirësuar""",100,102,false,false,true
"""trial_9""","""page_9""",4519,9507470,102,646,"""r""",34,true,130,"""nxjerrjen""",101,107,false,false,true
"""trial_9""","""page_9""",4520,9507956,107,682,""" """,35,true,343,""" """,102,106,false,true,true


In [18]:
#  annotate skipped words based on fixation table and all tokens
words_with_skip = mark_skipped_tokens(all_tokens, fixation_table)

In [19]:
# calculate word-level reading measures
word_level_table = build_word_level_table(
    words=words_with_skip,
    fix=fixation_table,
)

/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/preprocessing/metrics/reading_measures.py:250: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  pl.DataFrame(


In [27]:
with pl.Config(tbl_rows=50):
    print(word_level_table.filter(pl.col("page") == "page_1").select([
        "word_idx", "word", "skipped", "FPF", "TFC", "SL_in", "RPD_inc", "RBRT"
    ]))

shape: (109, 8)
┌──────────┬────────────┬─────────┬─────┬─────┬───────┬─────────┬──────┐
│ word_idx ┆ word       ┆ skipped ┆ FPF ┆ TFC ┆ SL_in ┆ RPD_inc ┆ RBRT │
│ ---      ┆ ---        ┆ ---     ┆ --- ┆ --- ┆ ---   ┆ ---     ┆ ---  │
│ i64      ┆ str        ┆ i8      ┆ i8  ┆ u32 ┆ i64   ┆ i64     ┆ i64  │
╞══════════╪════════════╪═════════╪═════╪═════╪═══════╪═════════╪══════╡
│ 0        ┆ Mali       ┆ 0       ┆ 1   ┆ 3   ┆ 0     ┆ 115     ┆ 115  │
│ 1        ┆ Magjik     ┆ 0       ┆ 1   ┆ 2   ┆ 1     ┆ 1078    ┆ 557  │
│ 2        ┆ -          ┆ 1       ┆ 0   ┆ 0   ┆ 0     ┆ 0       ┆ 0    │
│ 3        ┆ Dy         ┆ 1       ┆ 0   ┆ 0   ┆ 0     ┆ 0       ┆ 0    │
│ 4        ┆ fjalë      ┆ 0       ┆ 1   ┆ 2   ┆ 3     ┆ 273     ┆ 273  │
│ 5        ┆ Hyrjeje    ┆ 0       ┆ 1   ┆ 2   ┆ 1     ┆ 450     ┆ 450  │
│ 6        ┆ Historia   ┆ 0       ┆ 1   ┆ 2   ┆ 1     ┆ 251     ┆ 251  │
│ 7        ┆ për        ┆ 0       ┆ 1   ┆ 2   ┆ 1     ┆ 146     ┆ 146  │
│ 8        ┆ Hans       ┆ 0       ┆

## The END


	-- data collection folder
	---- ...
	---- fixations
	---- saccades(?)
	---- reading_measures
	---- raw_data (i.e. gaze sample csv)